    # Web-Scraping

In [ ]:
# scraped all table data without getting httpconnectionpool error
# season_standings and scores + extra tables + player_stats
# 549 tables + 36 tables
from dotenv import load_dotenv
load_dotenv(override=True)
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from webdriver_manager.microsoft import EdgeChromiumDriverManager
import time

options = webdriver.EdgeOptions()
options.add_argument('headless')
driver = webdriver.Edge(service=EdgeService(EdgeChromiumDriverManager().install()), options=options)

from web_scraping import get_soccer_league_season_standings, get_soccer_league_scores, get_soccer_leagues_ids, get_player_season_stats2

current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

leagues = {
    'premier_league': list(range(1982, 2000)),
    'ligue_1': list(range(1989, 2000)),
    'serie_a': list(range(1994, 2000)),
    'la_liga': list(range(1993, 2000)),
    'fussball-bundesliga': list(range(1988, 2000)),
    'champions-league': list(range(1992, 2000)),

}

for league, years in leagues.items():
    for year in years:
        print(year)
        get_soccer_league_season_standings(league, year, driver)
        get_soccer_league_scores(league, year, driver)
        try:
            for soccer_league_id in soccer_league_ids:
                if soccer_league_id["league_name"] == "Fußball-Bundesliga" or soccer_league_id["league_name"] == "Big 5 European Leagues Combined":
                    continue
                get_player_season_stats2(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"], driver, year)

                time.sleep(3)

        except Exception as e:
            print("not possible")
            print(e)
            pass

driver.quit()


1982
Saved soccer_premier_league_results1982-198391_overall_1982.csv
Saved soccer_premier_league_results1982-198391_home_away_1982.csv
Saved soccer_premier_league_sched_1982-1983_9_1_1982.csv
Saved soccer_Premier-League-Stats_player_stats_1982.csv
not possible
'NoneType' object is not callable
1983
Saved soccer_premier_league_results1983-198491_overall_1983.csv
Saved soccer_premier_league_results1983-198491_home_away_1983.csv
Saved soccer_premier_league_sched_1983-1984_9_1_1983.csv
Saved soccer_Premier-League-Stats_player_stats_1983.csv
not possible
'NoneType' object is not callable
1984
Saved soccer_premier_league_results1984-198591_overall_1984.csv
Saved soccer_premier_league_results1984-198591_home_away_1984.csv
Saved soccer_premier_league_sched_1984-1985_9_1_1984.csv
Saved soccer_Premier-League-Stats_player_stats_1984.csv
not possible
'NoneType' object is not callable
1985
Saved soccer_premier_league_results1985-198691_overall_1985.csv
Saved soccer_premier_league_results1985-198691

In [ ]:
# zeigt den Fehler, wenn alle Tabellen von den player stats geladen werden sollen

from dotenv import load_dotenv
load_dotenv(override=True)
import os
from os.path import join
import time

from web_scraping import get_player_season_stats, get_soccer_leagues_ids, get_soccer_league_season_standings


current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

for year in range(1982, 2000):
    for soccer_league_id in soccer_league_ids:
        try:
            standings = get_soccer_league_season_standings(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])

        except Exception as e:
            if hasattr(e, 'message'):
                print(e.message)
            else:
                print ('not possible')
                print(e)
                continue


Saved Premier-League-Stats_results1982-198391_overall.csv
Saved Premier-League-Stats_results1982-198391_home_away.csv
not possible
HTTPConnectionPool(host='localhost', port=50449): Max retries exceeded with url: /session/ab91ca3ea5833bb3e544ebc5f907786c/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000273AB7C6EB0>: Failed to establish a new connection: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte'))
not possible
HTTPConnectionPool(host='localhost', port=50449): Max retries exceeded with url: /session/ab91ca3ea5833bb3e544ebc5f907786c/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000273AB80B070>: Failed to establish a new connection: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte'))
not possible
HTTPConnectionPool(host='localhost', port=50449): Max retries exceeded with url: /session/a

In [1]:
# zeigt wie die player stats gescraped werden (nur premier league), also alle tabellen und dann alle Tabellen mit dem gleichen Namen werden zusammengefügt
# 1 Jahr geht, aber mehr dann nicht

from dotenv import load_dotenv
load_dotenv(override=True)
import os
from os.path import join
import time

from web_scraping import get_soccer_leagues_ids, get_soccer_league_season_standings, get_soccer_teams_url, get_player_season_stats

current_season = 2022
soccer_league_ids = get_soccer_leagues_ids()

for year in range(1999,2000):
    print(year)
    try:
        for soccer_league_id in soccer_league_ids:
            if soccer_league_id["league_name"] == "Fußball-Bundesliga" or soccer_league_id["league_name"] == "Big 5 European Leagues Combined":
                continue

            # get player stats
            df = get_player_season_stats(year, current_season, soccer_league_id["league_id"], soccer_league_id["league_stats_name"])
            df.to_csv(join(os.environ["SportsTables"], "soccer", f"{soccer_league_id['league_name'].replace(' ','-')}_player_stats_{year}.csv"), index=False)

            time.sleep(3)
    except:
        print("not possible")
        continue

1999
Saved Premier-League-Stats_stats_standard_9_1999-2000.csv
Saved Premier-League-Stats_matchlogs_for_1999-2000.csv
Saved Premier-League-Stats_stats_keeper_9_1999-2000.csv
Saved Premier-League-Stats_stats_shooting_9_1999-2000.csv
Saved Premier-League-Stats_stats_playing_time_9_1999-2000.csv
Saved Premier-League-Stats_stats_misc_9_1999-2000.csv
Saved Premier-League-Stats_results1999-200091_overall_1999-2000.csv
Saved Premier-League-Stats_results1999-200091_home_away_1999-2000.csv
not possible


# Extract column headers from different tables

In [4]:
import json
from glob import glob
from dotenv import load_dotenv
load_dotenv(override=True)

import os
from os.path import join
import sys
import pandas as pd

metadata = {}

tables = ["player_stats", "nations", "results_home_away", "results _overall", "shed", "stats_squad_keeper_against","stats_squads_keeper_for", "stats_squads_misc_against", "stats_squad_misc_for", "stats_squads_playing_time_against", "stats_squads_playing_time_for", "stats_squads_shooting_against", "stats_squads_shooting_for", "stats_squads_standard_against", "stats_squads_standard_for"]

for table in tables:
    column_names = []
    for found_table in glob(join(os.environ["SportsTables"], "soccer", f"*{table}_*.csv")):
        column_names.extend(list(pd.read_csv(found_table).columns))
    
    column_names = list(set(column_names))
    metadata[table] = {}
    for column_name in column_names:
        metadata[table][column_name] = None
    
if os.path.isfile("metadata.json"):
    print("metadata.json exists already!!! ")
else:
    with open("metadata.json", "w") as f:
        json.dump(metadata, f)

# Extract valid header file

In [12]:
import json
from glob import glob
import os 
#os.environ["SPORTS_DB"] = "/ext/daten-wi/slangenecker/SportsTables"
os.environ["SPORTS_DB"] = "C:/Users/jwuel/DataspellProjects/SportsTables"
from os.path import join
import pandas as pd

def get_sportsDB_soccer_type_mappings():
    with open(join(os.environ["SPORTS_DB"], "Soccer", "Soccer-Relations.json"), "r") as f:
        type_mappings = json.load(f)["soccer-big5Leagues-Players-2021-2022.csv"]["columns"][0]
        return type_mappings


def get_all_sportsDB_soccer_tables(only_file_names:bool=False):
    result = glob(join(os.environ["SPORTS_DB"], "Soccer", "soccerPlayerScraping", "*.csv"))
    if only_file_names:
        return [table.split("/")[-1] for table in result]
    else:
        return result

def load_sportsDB_soccer_table(table_name:str, only_headers:bool=False):
    df = pd.read_csv(join(os.environ["SPORTS_DB"], "Soccer", "soccerPlayerScraping", table_name))
    if only_headers:
        return df.columns
    else:
        return df

In [11]:
col_name_to_sem_type = get_sportsDB_soccer_type_mappings()

results = {}
for table in get_all_sportsDB_soccer_tables(True):
    print(table)
    results[table.split(".csv")[0]] = {}
    col_names = load_sportsDB_soccer_table(table, True)
    for col_num, col_name in enumerate(col_names):
        try:
            results[table.split(".csv")[0]][f"column_{col_num}"] = { "semanticType": col_name_to_sem_type[col_name]}
        except:
            print(col_name)
    
with open(join(os.environ["SPORTS_DB"], "Soccer", "sportsDB_type_sportsDB.json"), "w") as f:
    json.dump(results, f)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/jwuel/DataspellProjects/SportsTables\\Soccer\\Soccer-Relations.json'